In [18]:
# import torch
# torch.cuda.is_available()
# torch.zeros(1).cuda()
# torch.version.cuda

In [19]:
import torch
import numpy as np

np . set_printoptions(precision=3)

a = [1, 2, 3]
b = np.array([4, 5, 6], dtype=np.int32)
t_a = torch.tensor(a)
t_b = torch.from_numpy(b)

print(t_a)
print(t_b)

t_ones = torch.ones(2, 3)
print(t_ones.shape)
print(t_ones)

rand_tensor = torch.rand(2, 3)
print(rand_tensor)

In [20]:
# Manipulation

t_a_new = t_a.to(torch.int64)
print(t_a_new.dtype)

t = torch.rand(3, 5)
t_tr = torch.transpose(t, 0, 1)
print(t.shape, ' --> ', t_tr.shape)

t = torch.zeros(30)
t_reshape = t.reshape(5, 6)
print(t_reshape.shape)

t = torch.zeros(1, 2, 1, 4, 1)
t_sqz = torch.squeeze(t, 2)
print(t.shape, ' --> ', t_sqz.shape)

In [21]:
# Math ops

torch.manual_seed(1)

t1 = 2 * torch.rand(5, 2) - 1
t2 = torch.normal(mean=0, std=1, size=(5, 2))

t3 = torch.multiply(t1, t2)
print('Multiply', t3)

t4 = torch.mean(t1, axis=0)
print('Mean', t4)

t5 = torch.matmul(t1, torch.transpose(t2, 0, 1))
print('Matmul', t5)

t6 = torch.matmul(torch.transpose(t1, 0, 1), t2)
print('Matmul2', t6)

norm_t1 = torch.linalg.norm(t1, ord=2, dim=1)
print('Norm', norm_t1)

In [22]:
# Split, stack, concatenate

torch.manual_seed(1)

t = torch.rand(6)
print(t)
t_splits = torch.chunk(t, 3)
[print(item.numpy()) for item in t_splits]

In [23]:
torch.manual_seed(1)
t = torch.rand(5)
t_splits = torch.split(t, split_size_or_sections=[3, 2])
[print(item.numpy()) for item in t_splits]

In [24]:
A = torch.ones(3)
B = torch.zeros(2)
C = torch.cat([A, B], axis=0)
C

In [25]:
A = torch.ones(3)
B = torch.zeros(3)
S = torch.stack([A, B], axis=1)
S

In [26]:
# Input pipelines

from torch.utils.data import DataLoader

t = torch.arange(6, dtype=torch.float32)
data_loader = DataLoader(t)

for item in data_loader:
    print(item)
    
data_loader = DataLoader(t, batch_size=3, drop_last=False)
for i, batch in enumerate(data_loader, 1):
    print(f'batch {i}:', batch)

In [27]:
from torch.utils.data import Dataset, TensorDataset

torch.manual_seed(1)
t_x = torch.rand([4, 3], dtype=torch.float32)
t_y = torch.arange(4)

class JointDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

joint_dataset = TensorDataset(t_x, t_y)

for example in joint_dataset:
    print('x: ', example[0], 'y: ', example[1])
    
# or
joint_dataset = TensorDataset(t_x, t_y)

In [28]:
torch.manual_seed(1)
data_loader = DataLoader(
    dataset=joint_dataset,
    batch_size=2,
    shuffle=True
)

for i, batch in enumerate(data_loader, 1):
    print(f'batch {i}:', '\nx:', batch[0], '\ny:', batch[1])

In [29]:
# for each iteration the elements within a batch are shuffled
for epoch in range(2):
    print(f'-- epoch {epoch + 1}')
    for i, batch in enumerate(data_loader, 1):
        print(f'batch {i}:', '\nx:', batch[0], '\ny:', batch[1])

In [30]:
# Creating a dataset from files

import pathlib
import matplotlib.pyplot as plt
import os
from PIL import Image

imgdir_path = pathlib.Path('cat_dog_images')
file_list = sorted([
    str(path) for path in imgdir_path.glob('*.jpg')
])
print(file_list)

fig = plt.figure(figsize=(10, 5))
for i, file in enumerate(file_list):
    img = Image.open(file)
    print('Image shape:', np.array(img).shape)
    ax = fig.add_subplot(2, 3, i+1)
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(img)
    ax.set_title(os.path.basename(file), size=15)
plt.tight_layout()
plt.show()

In [31]:
labels = [
    1 if 'dog' in os.path.basename(file) else 0 
    for file in file_list
]
labels

In [41]:
import torchvision.transforms as transforms


class ImageDataset(Dataset):
    def __init__(self, file_list, labels, transform=None):
        self.file_list = file_list
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = Image.open(self.file_list[idx])
        if self.transform is not None:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.labels)


img_height, img_width = 80, 120
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((img_height, img_width))
])

image_dataset = ImageDataset(file_list, labels, transforms)
fig = plt.figure(figsize=(10, 6))
for i, example in enumerate(image_dataset):
    ax = fig.add_subplot(2, 3, i + 1)
    ax.set_xticks([])
    ax.set_yticks([])
    img = example[0].numpy().transpose((1, 2, 0))
    img = img.clip(0, 1)
    ax.imshow(img)
    ax.set_title(f'{example[1]}', size=15)

plt.tight_layout()
plt.show()

In [25]:
import torchvision

image_path = '../datasets/'

celeba_dataset = torchvision.datasets.CelebA(
    image_path,
    split='train',
    target_type='attr',
    download=False
)

In [ ]:
image_path = '../datasets/'

# ??
mnist_dataset = torchvision.datasets.MNIST(
    image_path, 
    'train',
    download=True
)

In [33]:
import torch
from itertools import islice
import matplotlib.pyplot as plt

assert isinstance(mnist_dataset, torch.utils.data.Dataset)

example = next(iter(mnist_dataset))
print(example)

fig = plt.figure(figsize=(15, 6))
for i, (image, label) in islice(enumerate(mnist_dataset), 10):
    ax = fig.add_subplot(2, 5, i + 1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(image, cmap='gray_r')
    ax.set_title(f'{label}', size=15)
plt.show()